In [1]:
import os
file_list = os.listdir('data/dataset/dataset/train')
file_list

['中国银行保险监督管理委员会 工业和信息化部 国家发展和改革委员会 财政部 中国人民银行 国家市场监督管理总局联合发布《关于进一步规范信贷融资收费 降低企业融资综合成本的通知》.txt',
 '优化营商环境更好服务市场主体工作方案.txt',
 '关于印发《关于进一步优化京津口岸营商环境促进跨境贸易便利化若干措施的公告》的通知.txt',
 '关于启动开展2020年度北京市专业技术资格评价工作的通知.txt',
 '关于启用初级注册安全工程师职业资格电子证书的通知.txt',
 '关于天津港口支持企业克服疫情影响实现复工复产若干措施的通知.txt',
 '关于推进纳税缴费便利化改革优化税收营商环境若干措施的通知.txt',
 '关于本市道路货物运输驾驶员从业资格考试制度改革的通告.txt',
 '关于深化京津口岸营商环境改革进一步促进跨境贸易便利化若干措施的公告.txt',
 '关于进一步推进工业产品生产许可证制度改革 加强事中事后监管有关工作的通知.txt',
 '北京市交通委员会 北京市公安局公安交通管理局关于简化公路养护工程开工条件的通知.txt',
 '北京市交通委员会关于印发《北京市公路养护工程管理实施办法》的通知.txt',
 '北京市交通委员会关于印发《北京市公路养护工程项目招标投标管理办法》的通知.txt',
 '北京市交通委员会关于印发《北京市公路工程招标投标活动投诉处理管理办法(试行)》的通知.txt',
 '北京市交通委员会关于持续优化公路养护工程领域营商环境的通知.txt',
 '北京市人力资源和社会保障局 北京市财政局 中国人民银行营业管理部关于印发《北京市创业担保贷款借款人资格认定细则》的通知.txt',
 '北京市人力资源和社会保障局 北京市财政局 国家税务总局北京市税务局关于阶段性降低本市工伤保险费率的通告.txt',
 '北京市人力资源和社会保障局关于启用北京市电子职称证书的通知.txt',
 '北京市人民政府办公厅关于印发《北京市进一步优化营商环境更好服务市场主体实施方案》的通知.txt',
 '北京市人民政府办公厅关于进一步做好电子营业执照推广应用工作的通知.txt',
 '北京市人民政府办公厅转发市公安局《关于电子印章管理工作意见》的通知.txt',
 '北京市企业投资项目“区域评估+标准地+承诺制+政府配套”改革试点工作方

In [2]:
target_path = 'data/dataset/dataset/train/'
source_path = 'data/dataset/dataset_augment_synonym/train/'
for file in file_list:
    with open(source_path+file,'r',encoding='utf-8') as infile:
        with open(target_path+file,'w',encoding='utf-8') as outfile:
            for line in infile.readlines():
                outfile.write(line)

In [50]:
# 读取分类
import os
mark_path = 'data/new_data/mark_correction/'

marks_list = ['mark1/','mark2/']
real_dict = {}
marks = {}
for mk in marks_list:
    sentences_titile = {}
    for file in os.listdir(mark_path+mk):
        with open(mark_path+mk+file,'r',encoding='utf-8') as infile:
            for line in infile.readlines():
                set = line.strip().split('\t')
                sentences_titile[set[1]] = int(set[4])
                real_dict[set[1]] = int(set[2])
    marks[mk.replace('/','')] = sentences_titile

sentences_titile = {}
for file in os.listdir(mark_path+'mark3/'):
    with open(mark_path+'mark3/'+file,'r',encoding='utf-8') as infile:
        for line in infile.readlines():
            set = line.strip().split('\t')
            sentences_titile[set[1]] = int(set[2])
marks['mark3'] = sentences_titile

In [47]:
for mark in marks:
    print(mark)


mark1
mark2
mark3


In [51]:
from sklearn.metrics import confusion_matrix
def caculate_kappa(matrix):
    N, p0 = 0, 0
    ai_r, ai_c = [], [0]*len(matrix)
    for i in range(len(matrix)):
        N += sum(matrix[i])
        p0 += matrix[i][i]
        ai_r.append(sum(matrix[i]))
        for j in range(len(matrix[i])):
            ai_c[j] += matrix[i][j]
    p0 /= N
    pe = 0
    for i in range(len(ai_r)):
        pe += ai_r[i] * ai_c[i]
    pe /= (N*N)
    result = (p0-pe)/(1-pe)
    return result

kappa_result = []
for mark in marks:
    reallist, predlist = [], []
    for sentence in marks[mark]:
        predlist.append(marks[mark][sentence])
        reallist.append(real_dict[sentence])
    matrix = confusion_matrix(y_true=reallist, y_pred=predlist)
    kappa_result.append(caculate_kappa(matrix))

print(kappa_result)

average = sum(kappa_result) / len(kappa_result)
average

[0.9427182975035185, 0.8882084361284152, 0.9798550776056917]


0.9369272704125419

In [2]:
import torch
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained('model/bert') # 包含上面三个文件的文件夹目录
model = BertModel.from_pretrained('model/bert')
input_ids = torch.tensor(tokenizer.encode("自然语言处理")).unsqueeze(0)  # Batch size 1
print(input_ids,input_ids.shape)
outputs = model(input_ids)
# last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
sequence_output = outputs[0]
pooled_output = outputs[1]
print(sequence_output)
print(sequence_output.shape)    ## 字向量
print(pooled_output.shape)      ## 句向量

Some weights of the model checkpoint at model/bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 5632, 4197, 6427, 6241, 1905, 4415,  102]]) torch.Size([1, 8])
tensor([[[-0.5152, -0.0859,  0.8517,  ..., -0.0063, -0.4380, -0.0603],
         [ 0.2122, -0.4674,  0.7332,  ..., -0.0799, -0.2016,  0.6670],
         [ 0.1858, -0.6106, -0.0027,  ..., -0.2438, -0.1905,  0.4633],
         ...,
         [ 0.1136,  0.2130,  1.3360,  ..., -0.1908,  0.3598,  0.0100],
         [-0.2338, -0.1829,  0.8073,  ..., -0.1682,  0.5623,  0.1590],
         [-0.1665,  0.2620,  1.2459,  ..., -0.2552, -0.1596,  0.0035]]],
       grad_fn=<NativeLayerNormBackward>)
torch.Size([1, 8, 768])
torch.Size([1, 768])


In [3]:
print(sequence_output.shape)
input_ids.shape

torch.Size([1, 8, 768])


torch.Size([1, 8])

In [6]:
input = ['你好','我是中国人','你是哪个国家的人']

from torch import nn

input1 = []
contents_len = [2,5,8]
for i in input:
    temp = torch.tensor(model(torch.tensor(tokenizer.encode(i)).unsqueeze(0))[0]).squeeze(0)
    input1.append(temp)
    print(temp.shape)
    # input1.append(self.embedding(i))
# print(input1)

x_pad = nn.utils.rnn.pad_sequence(input1, batch_first=True, padding_value=0)
x_pack = nn.utils.rnn.pack_padded_sequence(x_pad, contents_len, batch_first=True)

D:\program files\Anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([4, 768])
torch.Size([7, 768])
torch.Size([10, 768])


RuntimeError: `lengths` array must be sorted in decreasing order when `enforce_sorted` is True. You can pass `enforce_sorted=False` to pack_padded_sequence and/or pack_sequence to sidestep this requirement if you do not need ONNX exportability.

In [35]:
# 统计图中的边、节点等信息
import json
import ast

information = {}
# 统计描述性事件、概括性事件的数量
with open('data/new_data/real_triple/type_list.json','r',encoding='utf-8') as infile:
    type_info = json.load(infile)

general_count, describe_count, entity_set, event_edge_count, entity_edge_count, type_set = 0, 0, set(), 0, 0, set()
for i in type_info:
    if type_info[i] == '概括性措施':
        general_count += 1
    elif type_info[i] == '描述性措施':
        describe_count += 1
    else:
        type_set.add(type_info[i])

# 统计节点的数量
triple_file_list = ['describe.txt','institution_att.txt','release.txt','synergy.txt','result.txt']
for triple_file in triple_file_list:
    with open('data/new_data/real_triple/'+triple_file,'r',encoding='utf-8') as infile:
        for line in infile.readlines():
            sets = ast.literal_eval(line.strip())
            flag = False
            for te in [sets[0],sets[2]]:
                entity_type = type_info.get(te,'entity')
                if entity_type == '描述性措施' or entity_type == '概括性措施':
                    flag = True
                else:
                    entity_set.add(te)
                    continue
            if flag:
                event_edge_count += 1
            else:
                entity_edge_count += 1
entity_set.remove('')

information['general'] = general_count
information['describe'] = describe_count
information['event'] = general_count + describe_count
information['entity'] = len(entity_set)
information['event_edge'] = event_edge_count
information['entity_edge'] = entity_edge_count
information['edge'] = event_edge_count + entity_edge_count
information['type_count'] = len(type_set)

information

{'general': 1009,
 'describe': 2466,
 'event': 3475,
 'entity': 1003,
 'event_edge': 3655,
 'entity_edge': 2537,
 'edge': 6192,
 'type_count': 40}

In [3]:
from config.dataset_loader import dataset
data_dir = 'data/dataset/dataset_augment_synonym/'

dt = dataset(data_dir, max_length=128, word_or_char='char')
train_data = dt.to_pack_sequence_string(dt.train_data)
test_data = dt.to_pack_sequence_string(dt.test_data)
dev_data = dt.to_pack_sequence_string(dt.dev_data)

2260
{'<pad>': 0, '，': 1, '、': 2, '的': 3, '人': 4, '市': 5, '业': 6, '法': 7, '理': 8, '行': 9, '务': 10, '工': 11, '(': 12, ')': 13, '关': 14, '和': 15, '一': 16, '管': 17, '定': 18, '税': 19, '公': 20, '2': 21, '作': 22, '通': 23, '0': 24, '政': 25, '办': 26, '实': 27, '用': 28, '规': 29, '程': 30, '1': 31, '标': 32, '不': 33, '京': 34, '企': 35, '应': 36, '件': 37, '化': 38, '进': 39, '事': 40, '信': 41, '审': 42, '提': 43, '第': 44, '发': 45, '合': 46, '条': 47, '项': 48, '产': 49, '部': 50, '案': 51, '在': 52, '北': 53, '中': 54, '机': 55, '制': 56, '等': 57, '申': 58, '对': 59, '为': 60, '以': 61, '有': 62, '保': 63, '服': 64, '证': 65, '局': 66, '设': 67, '建': 68, '资': 69, '可': 70, '营': 71, '时': 72, '民': 73, '施': 74, '动': 75, '要': 76, '记': 77, '及': 78, '国': 79, '当': 80, '登': 81, '监': 82, '交': 83, '场': 84, '电': 85, '单': 86, '开': 87, '会': 88, '本': 89, '门': 90, '院': 91, '请': 92, '查': 93, '区': 94, '分': 95, '十': 96, '于': 97, '或': 98, '经': 99, '“': 100, '二': 101, '”': 102, '全': 103, '知': 104, '主': 105, '\u3000': 106, '员': 107, '体': 108, '各': 

Some weights of the model checkpoint at model/bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/756 [00:00<?, ?it/s]D:\file\python files\Measures_detect\config\dataset_loader.py:277: UserWarning: To copy construct from a tensor, it is reco

In [ ]:
import pickle
pickle.dump(train_data,open('logs/pickle/train_bert.pickle','wb'))
pickle.dump(test_data,open('logs/pickle/test_bert.pickle','wb'))
pickle.dump(dev_data,open('logs/pickle/dev_bert.pickle','wb'))

D:\program files\Anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import paddlehub as hub
hub.TextClassifierTask().finetune_and_eval()

In [9]:
# 输入一个正数
# n = int(input())
# 输入一个空格分割的一个list
list1 = list(map(int,input().split()))

# list1 = sorted(list1)
st = set()
seen = set()
for i in list1:
    if i in seen:
        continue
    else:
        sc = []
        for j in st:
            if i % j == 0:
                sc.append(j)
                seen.add(j)
        st.add(i)
        for number in sc:
            st.remove(number)
st

{4, 5}